In [2]:
from functions.utils_lrp import *
os.environ['CUDA_VISIBLE_DEVICES'] = "1"  # Set the GPU card to use
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Or '3' for FATAL logs only
np.set_printoptions(suppress=True)
physical_devices = tf.config.list_physical_devices('GPU')
for gpu_instance in physical_devices:
    tf.config.experimental.set_memory_growth(gpu_instance, True)
import yaml
# Set global parameters
lookback = 3
nr = 21
city = 'Milan'
nr2 = 2
k = 4
random_flag = False
# Directories
output_directory = '../../../../../oracle-data/serly/Scalable_dnn/PerBS/'
cluster_directory = '../../../../../oracle-data/serly/Scalable_dnn/cluster/'
model_deepcog_miMo = '../../../../../oracle-data/serly/Scalable_dnn/Trained_models/DeepCOG_miMo/'
model_deepcog_original = '../../../../../oracle-data/serly/Scalable_dnn/Trained_models/DeepCOG_original/'
main_dir = f'../../../../../oracle-data/serly/Scalable_dnn/MAE_plots/DeepCOG_miMo_vs_MiMo/'

In [8]:
cells_dir = f'../../../../../oracle-data/serly/Scalable_dnn/LRP_scores/mimo_model/{city}/K_{k}/'
ids = np.load(os.path.join(cells_dir, f'sorted_cells_{0}.npy'))
ids

array([5370, 5470, 5170, 5369, 5450, 5169, 5250, 5469, 5550, 5253, 5168,
       5270, 5266, 5265, 5150, 5366, 5070, 4763, 5466, 5067, 5069, 4851,
       5551, 4867, 4969, 4156, 5350, 5365, 5364, 5357, 4850, 4155, 5066,
       4868, 4764, 5451, 5167, 5065, 4752, 4866, 4852, 5554, 5356, 5153,
       5052, 5569, 5555, 4164, 5570, 4970, 5456, 5465, 4154, 4966, 5553,
       5552, 5652, 4550, 4165, 4952, 4157, 5556, 4151, 4163, 5565, 4152,
       4951, 4669, 4066, 4770, 4865, 4150, 4651, 4153, 5653, 5455, 4864,
       5464, 4470, 5051, 4070, 4652, 5050, 4067, 5351, 4162, 4161, 4065,
       5650, 4069, 5165, 4062, 4068, 5651, 4670, 4551, 4751, 4170, 4061,
       4063, 5564, 4159, 4053, 4059, 4060, 4869, 4052, 4357, 4668, 4169,
       5457, 4965, 4250, 4358, 5768, 5755, 4570, 5751, 5764, 4058, 4166,
       4251, 5452, 4064, 4158, 4750, 5763, 5669, 5770, 4054, 4259, 4258,
       5666, 5454, 5756, 5668, 4469, 4051, 4765, 4252, 4369, 4368, 5670,
       4453, 4950, 4370, 4050, 5557, 5166, 4452, 42

In [ ]:
def initialize_relevance(output, target_class=None):
    """
    Initialize relevance at the output layer.
    """
    relevance = np.zeros_like(output)
    if target_class is not None:
        relevance[:, target_class] = output[:, target_class]
    else:
        relevance = output / np.sum(output, axis=1, keepdims=True)  # Distribute relevance equally
    return tf.convert_to_tensor(relevance, dtype=tf.float32)

def compute_lrp(model, inputs, relevance_output):
    """
    Compute Layer-Wise Relevance Propagation (LRP).
    """
    relevance = tf.convert_to_tensor(relevance_output, dtype=tf.float32)

    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Dense):
            weights, _ = layer.get_weights()
            relevance = tf.matmul(relevance, tf.transpose(weights))
        elif isinstance(layer, tf.keras.layers.Flatten):
            relevance = tf.reshape(relevance, (-1, *layer.input_shape[1:]))
        elif isinstance(layer, tf.keras.layers.Conv3D):
            kernel, _ = layer.get_weights()
            strides = [1, *layer.strides, 1]
            padding = layer.padding.upper()
            output_shape = [inputs.shape[0]] + list(layer.input_shape[1:])
            relevance = tf.nn.conv3d_transpose(
                relevance,
                kernel,
                output_shape=output_shape,
                strides=strides,
                padding=padding
            )
        elif isinstance(layer, tf.keras.layers.Dropout) or isinstance(layer, tf.keras.layers.InputLayer):
            continue
        else:
            raise NotImplementedError(f"Unsupported layer type: {type(layer)}")

    return relevance


In [ ]:
cells = get_rows_Milan(5060, nr)
length = 1780
cluster_counts = np.load(os.path.join(cluster_directory, f'clusters_{city}_{k}.npy'))
clustered_cells = {i: [] for i in range(k)}

# Populate the cluster cells dictionary
for idx, cluster_label in enumerate(cluster_counts):
    cell_id = cells[idx]
    clustered_cells[cluster_label].append(cell_id)

results = []
for cluster_label, cluster_cells in clustered_cells.items():
    local_nr2 = nr2
    local_num_cells = nr2 * nr2
    total_cells = len(cluster_cells)

    # Adjust if the number of cells exceeds available cells
    if local_num_cells > total_cells:
        largest_square = int(math.floor(math.sqrt(total_cells))) ** 2
        local_num_cells = largest_square
        local_nr2 = int(math.sqrt(local_num_cells))
        print(f"Number of cells for cluster {cluster_label} in K:{k} is set to {local_nr2} (nearest square number).")
    random.seed(42)
    selected_cells = (random.sample(cluster_cells, local_num_cells) if random_flag else 
                        np.load(os.path.join(cluster_directory, f'closest_to_centroid/closest_bs_{city}_{k}_{cluster_label}_100.npy')).tolist()[:local_num_cells])

    # Get the directory of the current script file
    # config_path = f'config{local_nr2}.yml'
    # with open(config_path, 'r') as file:
    #     conf = yaml.safe_load(file)
    # Prepare data
    X_test, y_test, y_scalers = preprocess_cluster_data(selected_cells, cluster_cells, output_directory, city, lookback, local_nr2, data_type="test")
    X_test_original, y_test_original, y_scalers_original = preprocess_cluster_data(cells, cells, output_directory, city, lookback, nr, data_type="test")

    model_miMo_path = os.path.join(model_deepcog_miMo, f'random_selection_{random_flag}/{city}/k_{k}/cluster_{cluster_label}_size_{local_num_cells}.h5')
    model_original_path = os.path.join(model_deepcog_original, f'{city}/deepcog_size_{nr}.h5')
    model_miMo = load_model_deepcog(model_miMo_path)
    predicted_miMo = model_miMo.predict(X_test)

    model_original = load_model_deepcog(model_original_path)
    predicted_original = model_original.predict(X_test_original)

In [ ]:
# Forward pass: Get predictions
output = model_original(X_test_original)

# Initialize relevance at the output layer (focus on class 0)
relevance_output = np.zeros_like(output.numpy())
target_class = 0  # Class to explain
relevance_output[:, target_class] = output[:, target_class]

# Compute LRP
lrp_map = compute_lrp(model_original, X_test_original, relevance_output)

# Print results
print("LRP Map Shape:", lrp_map.shape)  # Should match input shape (batch, lookback, size, size, 1)


In [ ]:
# Forward pass to get predictions
output = model_original(X_test_original)

# Initialize relevance (focus on class 0)
relevance_output = initialize_relevance(output.numpy(), target_class=0)

# Compute LRP
lrp_map = compute_lrp(model_original, X_test_original, relevance_output)

# Print shape of relevance map
print("LRP Map Shape:", lrp_map.shape)  # Should match input shape (10, 3, 2, 2, 1)

# Visualize relevance for the first sample
import matplotlib.pyplot as plt

# Aggregate relevance over the temporal dimension (lookback)
aggregated_lrp_map = np.sum(lrp_map.numpy(), axis=1)
print("Aggregated LRP Map Shape:", aggregated_lrp_map.shape)  # Should match input shape (10, 2, 2, 1)
plt.imshow(aggregated_lrp_map[0, :, :, 0], cmap='hot', interpolation='nearest')
plt.colorbar()
plt.title("Relevance Map for First Sample")
plt.show()


In [ ]:
# compute the costs (deepcog cost and mae cost) for miMo and original deepcog and save them in a csv files
import multiprocessing
from multiprocessing import Pool
from functions.utils_lrp import *


# Set multiprocessing start method
multiprocessing.set_start_method('spawn', force=True)
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"  # Set the GPU card to use
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Or '3' for FATAL logs only
np.set_printoptions(suppress=True)
physical_devices = tf.config.list_physical_devices('GPU')
for gpu_instance in physical_devices:
    tf.config.experimental.set_memory_growth(gpu_instance, True)

# Set global parameters
lookback = 3
nr = 21

# Directories
main_directory = '../../../../oracle-data/serly/Scalable_dnn/'
dataset_directory = '../../../../oracle-data/serly/Scalable_dnn/PerBS/'
cluster_directory = '../../../../oracle-data/serly/Scalable_dnn/cluster/'
model_deepcog_mimo = '../../../../oracle-data/serly/Scalable_dnn/Trained_models/DeepCOG_MiMo_clusters/'
output_directory = '../../../../oracle-data/serly/Scalable_dnn/MAE_plots/DeepCOG_miMo_vs_MiMo/'

# Function to process a single parameter combination
def process_combination(params):
    K, nr2, city = params
    
    cells = get_rows_Milan(5060, nr)
    cluster_counts = np.load(os.path.join(cluster_directory, f'clusters_{city}_{K}.npy'))
    clustered_cells = {i: [] for i in range(K)}

    # Populate the cluster cells dictionary
    for idx, cluster_label in enumerate(cluster_counts):
        cell_id = cells[idx]
        clustered_cells[cluster_label].append(cell_id)

    for cluster_label, cluster_cells in clustered_cells.items():
        
        # load the lrp scores
        lrp_dir = os.path.join(main_directory, f'LRP_scores/mimo_model/{city}/K_{K}')
        lrp_scores = np.load(os.path.join(lrp_dir, f'cluster_{cluster_label}_size_{total_cells}.npy'))


        # Print results
        print("LRP Map Shape:", lrp_map.shape)  # Should match input shape (batch, lookback, size, size, 1)


# Prepare combinations for multiprocessing
Ks = [2, 3, 4, 5, 6, 10, 15, 20]
nr2_values = [2, 3, 4, 5, 6, 7]
cities = ['Milan']
combinations = [(K, nr2, city) for K in Ks for nr2 in nr2_values for city in cities]

# Run multiprocessing
if __name__ == '__main__':
    with Pool(100) as p:
        all_results = p.map(process_combination, combinations)


In [ ]:
def process_lrp_scores(Ks, city, lookback=3, nr=21):
    """
    Process LRP scores for a specific city and multiple cluster counts (Ks).
    
    Args:
        Ks (list[int]): List of cluster counts to process.
        city (str): City name ('EUMA' or 'Milan').
        lookback (int): Lookback window for the model (default: 3).
        nr (int): Number of rows/columns in the grid (default: 21).
    """
    # Directories
    main_directory = '../../../../../oracle-data/serly/Scalable_dnn/'
    dataset_directory = f'{main_directory}PerBS/'
    cluster_directory = f'{main_directory}cluster/'
    model_deepcog_mimo = f'{main_directory}Trained_models/DeepCOG_MiMo_clusters/'

    cells = get_rows_Milan(5060, nr)

    for K in Ks:
        print(f"Processing city: {city}, K = {K}")
        
        output_directory = f'{model_deepcog_mimo}{city}/k_{K}/'
        lrp_dir = os.path.join(main_directory, f'LRP_scores/mimo_model/{city}/K_{K}')

        # Load the cluster assignments
        cluster_counts = np.load(cluster_directory + f'clusters_{city}_{K}.npy')

        # Initialize a dictionary to store the cell IDs for each cluster
        clustered_cells = {i: [] for i in range(K)}

        # Categorize the cells into clusters
        for idx, cluster_label in enumerate(cluster_counts):
            cell_id = cells[idx]
            clustered_cells[cluster_label].append(cell_id)

        # Process each cluster
        for cluster_label in clustered_cells:
            # Get the cell IDs for the current cluster
            cluster_cells = clustered_cells.get(cluster_label, [])
            if not cluster_cells:
                print(f"No cells found for Cluster {cluster_label}")
                continue

            total_cells = len(cluster_cells)  # Total number of cells in this cluster
            lrp_scores_path = os.path.join(lrp_dir, f'cluster_{cluster_label}_size_{total_cells}.npy')
            cell_ids_path = os.path.join(output_directory, f'cluster_cells_{cluster_label}.npy')

            if not os.path.exists(lrp_scores_path) or not os.path.exists(cell_ids_path):
                print(f"LRP scores or cell IDs missing for Cluster {cluster_label}")
                continue

            # Load LRP scores and cell IDs
            lrp_scores = np.load(lrp_scores_path)
            print(lrp_scores.shape)
            cell_ids = np.load(cell_ids_path)


            # Process LRP scores
            # remove the 4-dimension only
            my_scores = np.squeeze(lrp_scores, axis=-1)
            my_scores = my_scores[:, 2, :, :]  # Extract specific time step dimension
            my_scores = np.mean(my_scores, axis=0)  # Average over time
            my_scores = my_scores.flatten()  # Flatten into 1D array

            # Sort cells in descending order of scores
            sorted_indices = np.argsort(my_scores)[::-1]
            cells_sorted = cell_ids[sorted_indices]

            # Save the sorted cells
            sorted_path = os.path.join(lrp_dir, f'sorted_cells_{cluster_label}.npy')
            np.save(sorted_path, cells_sorted)


Ks = [2, 3, 4, 5, 6, 10, 15, 20]
process_lrp_scores(Ks=Ks, city='Milan')
